<a href="https://colab.research.google.com/github/dev-tyta/dogs_vs_cats/blob/main/models/dogs_cats_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Major

/content/drive/MyDrive/Major


In [4]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Major"

In [5]:
!kaggle competitions download -c dogs-vs-cats

dogs-vs-cats.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
!unzip dogs-vs-cats.zip

Archive:  dogs-vs-cats.zip
replace sampleSubmission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:

!unzip train.zip
!unzip test1.zip

Archive:  train.zip
replace train/cat.0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
  inflating: train/dog.0.jpg         
  inflating: train/dog.1.jpg         
  inflating: train/dog.10.jpg        
  inflating: train/dog.100.jpg       
  inflating: train/dog.1000.jpg      
  inflating: train/dog.10000.jpg     
  inflating: train/dog.10001.jpg     
  inflating: train/dog.10002.jpg     
  inflating: train/dog.10003.jpg     
  inflating: train/dog.10004.jpg     
  inflating: train/dog.10005.jpg     
  inflating: train/dog.10006.jpg     
  inflating: train/dog.10007.jpg     
  inflating: train/dog.10008.jpg     
  inflating: train/dog.10009.jpg     
  inflating: train/dog.1001.jpg      
  inflating: train/dog.10010.jpg     
  inflating: train/dog.10011.jpg     
  inflating: train/dog.10012.jpg     
  inflating: train/dog.10013.jpg     
  inflating: train/dog.10014.jpg     
  inflating: train/dog.10015.jpg     
  inflating: train/dog.10016.jpg     
  inflating: train/dog.10017.jpg     
  i

In [8]:
import tensorflow as tf
import os
from pathlib import Path
from tqdm import tqdm

In [9]:
import pandas as pd
import numpy as np

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import Sequential, layers

In [11]:
train_dir= "train"
test_dir= "test1"

In [12]:
train_df = pd.DataFrame({'image_name':os.listdir(train_dir)})
train_df['label'] =train_df['image_name'].apply(lambda x: x.split('.')[0])
train_df

,image_name,label
0,cat.9090.jpg,cat
1,cat.9095.jpg,cat
2,cat.9085.jpg,cat
3,cat.9100.jpg,cat
4,cat.9069.jpg,cat
...,...,...
24998,dog.9995.jpg,dog
24999,dog.9996.jpg,dog
25000,dog.9997.jpg,dog
25001,dog.9998.jpg,dog


In [13]:
test_df = pd.DataFrame({'image_name':os.listdir(test_dir)})
test_df['label'] =test_df['image_name'].apply(lambda x: x.split('.')[0])
test_df

,image_name,label
0,11487.jpg,11487
1,11491.jpg,11491
2,11534.jpg,11534
3,11541.jpg,11541
4,11548.jpg,11548
...,...,...
12811,773.jpg,773
12812,766.jpg,766
12813,810.jpg,810
12814,764.jpg,764


In [16]:
# dog_path_train = os.path.join(train_dir, 'dog')
# os.mkdir(dog_path_train)
# dog_df_train = train_df[train_df.label=='dog']


In [17]:
# for n in tqdm(dog_df_train.image_name):
#     os.rename((os.path.join(train_dir, n)), (os.path.join(dog_path_train, n)))

In [18]:
# cat_path_train = os.path.join(train_dir, 'cat')
# os.mkdir(cat_path_train)
# cat_df_train = train_df[train_df.label=='cat']


In [19]:
# for n in tqdm(cat_df_train.image_name):
#     os.rename((os.path.join(train_dir, n)), (os.path.join(cat_path_train, n)))

In [20]:
train_datagen = ImageDataGenerator(rescale= 1./255, validation_split= 0.3)

In [21]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size= (150, 150),
    class_mode='binary',
    subset='training'
)

Found 8578 images belonging to 3 classes.


In [22]:
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size= (150, 150),
    class_mode= 'binary',
    subset= 'validation'
)

Found 3675 images belonging to 3 classes.


In [23]:
model = Sequential([
    layers.Conv2D(16, (3,3), activation= 'relu', input_shape =(150, 150, 3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(32, (3,3), activation= 'relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation= 'relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation= 'relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation= 'relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(512, activation= 'relu'),
    layers.Dense(1,activation= 'sigmoid'),
])

In [ ]:
model.summary()

In [ ]:
model.compile(loss= 'binary_crossentropy',
             optimizer = RMSprop(learning_rate= 0.001),
              metrics= ['accuracy']
             )

In [ ]:
class mycallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        if logs.get('val_accuracy')>=0.90:
            self.model.stop_training = True
            
callback = mycallback()

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples//50,#batch_size, 
    epochs = 30,
    verbose=1,
    validation_data = validation_generator,
    validation_steps = validation_generator.samples//50,#batch_size,
    callbacks=[callback]
)

In [ ]:
model.save("dogs_cats.h5")

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
import os

#new = Path.('\dogs-vs-cats\test1')

images = os.listdir(r'test1')
print(images)



In [ ]:
for i in images:
    print()
    # predicting images
    path = r'test1/' + i
    # print(path)
    img = image.load_img(path, target_size=(150, 150))
    x = image.img_to_array(img)
    x /= 255
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    print(classes[0])
    results = []
    if classes[0]>0.5:
        results.append('Dog')
    if classes[0]< 0.5:
        results.append('Cat')

In [ ]:
print(results)

In [ ]:
img = os.listdir(r'C:\Users\Hp 15\Documents\My_files\Image Classification\tmp')
print(img)

In [ ]:
for i in images:
    print()
    # predicting images
    path = r'C:\Users\Hp 15\Documents\My_files\Image Classification\tmp/' + i
    img = image.load_img(path, target_size=(150, 150))
    x = image.img_to_array(img)
    x /= 255
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    print(classes[0])
    results = []
    if classes[0]>0.5:
        results.append('Dog')
    else:
        results.append('Cat')

In [ ]:
print(results)